In [45]:
forbidden_genres = {'religion_self', 'romance_fantasy', 'real_estate', 'literature_western', 'sci_phys', 'love', 'romance_sf', 'adv_maritime', 'sci_state', '105', 'literature_world', 'poetry', 'performance', 'sf_humor', 'industries', 'management', 'sci_transport', 'sf_horror', 'reference', 'computers', 'fantasy_alt_hist', 'stock', 'religion_buddhism', 'sci_religion', 'adventure', 'literature_books', 'home_diy', 'sci_culture', 'comp_soft', 'literature_history', 'det_history', 'marketing', 'sci_philology ', 'det_espionage', 'literature_classics', 'love_sf', 'Эзотерика', 'literature_poetry', 'religion', 'home', 'antique_myths', 'antique_east', 'ref_guide',
'romance', 'travel_africa', 'humor_verse', 'personal_finance', 'ref_dict', 'sci_philology', 'Боевая фантастика', 'geo_guides', 'comp_hard', 'religion_rel', 'sci_math', 'literature_short', 'dramaturgy', 'sf_epic', 'popadanec', 'sf_history', 'prose', 'religion_budda', 'comp_www', 'sf_action', 'sci_psychology', 'health_sex', 'teens_literature',
'humor_anecdote', 'economics_ref', 'literature_fairy', 'det_hard', 'История', 'antique_russian', 'sci_chem', 'home_health', 'det_crime', 'home_pets', 'literature_adv', 'sci_philosophy', 'rel_boddizm', 'nonfiction', 'sf_cyberpunk', 'sci_tech', 'outdoors_travel', 'home_entertain', 'nonf_criticism', 'sf', 'sci_linguistic', 'nonfiction_politics', 'sf_fantasy', 'comp_osnet', 'det_police', 'prose_history            ', 'literature_antology', 'biogr_sports', 'sf_social', 'religion_esoterics', 'sci_medicine', 'travel_europe', 'Prose', 'literature_political', 'home_sport', 'sf_postapocalyptic', 'paper_work', 'design',
'thriller_mystery', 'Юмор', 'det_political', 'literature_women', 'outdoors_hunt_fish', 'sf_detective', 'people', 'sf_fantasy_city', 'periodic', 'love_detective', 'love_erotica', 'prose_history', 'global_economy', 'humor', 'other', 'entert_humor', 'prose_su_classics', 'love_contemporary', 'sci_cosmos', 'accounting', 'det_maniac', 'det_action', 'biogr_leaders', 'religion_east', 'sf_heroic', 'sci_business', 'det_irony', 'sci_juris', 'histor_military', 'proce', 'adv_indian', 'sf_fantasy               ', 'ref_ref', 'prose_classic', 'banking', 'history_russia', 'org_behavior', 'det_classic', 'literature', 'ref_encyc', 'science', 'adv_history',
'nonf_publicism', 'network_literature', 'antique_ant', 'home_cooking', 'sci_history', 'love_history', 'antique', 'history_europe', 'sci_biology', 'love_short', 'comp_programming', 'religion_spirituality', 'comp_db', 'job_hunting', 'prose_contemporary', 'economics', 'prose_military', 'home_crafts', 'sci_politics', 'thriller', 'adv_western', 'nonf_biography', 'romance_romantic_suspense', 'humor_prose', 'adv_geo', 'literature_war', 'prose_rus_classic', 'home_garden', 'antique_european', 'history_world', 'prose_counter', 'sf_space', 'popular_business', 'detective', 'chris_pravoslavie', 'home_sex', 'adv_animal',
'N/K', 'adv_all', 'adventure                ', 'art', 'biogr_arts', 'biogr_historical', 'biogr_professionals', 'biography', 'biz_beogr', 'biz_economics', 'biz_management', 'business', 'chris_orthodoxy', 'epistolary_fiction', 'family', 'family_health', 'family_parenting', 'family_pregnancy', 'family_relations', 'health', 'health_alt_medicine', 'health_men', 'health_nutrition', 'health_psy', 'health_rel', 'health_self_help', 'health_sex', 'health_women', 'historical_fantasy', 'history_ancient', 'history_asia', 'history_australia', 'history_usa', 'horror', 'horror_fantasy', 'horror_occult', 'horror_usa', 'horror_vampires',
'literature_20', 'literature_drama', 'literature_essay', 'literature_rus_classsic', 'literature_sea', 'literature_su_classics', 'mystery', 'nonf_military', 'nonfiction_edu', 'nonfiction_folklor', 'nonfiction_law', 'nonfiction_philosophy', 'nonfiction_social_sci',
'nonfiction_social_work', 'nonfiction_true_accounts', 'outdoors_conservation', 'outdoors_fauna', 'outdoors_nature_writing', 'professional_law', 'prose_root', 'prose_su_classic', 'religion_hinduism', 'religion_occult', 'romance_historical', 'romance_multicultural', 'sci_build', 'sci_economy', 'sci_history История', 'science_archaeology', 'science_biolog', 'science_earth', 'science_history_philosophy', 'science_medicine', 'science_psy', 'sf_cyber_punk', 'sf_fantasy_irony', 'sf_history_avant', 'sf_irony', 'sf_mystic', 'sf_writing', 'small_business', 'sport', 'teens_health', 'teens_history', 'teens_literature', 'teens_social',
'thriller_police', 'travel', 'travel_asia', 'travel_guidebook_series', 'travel_lat_am', 'travel_spec', 'women_single', 'Биографии и мемуары', 'Интернет', 'Иронический детектив', 'Классическая проза', 'Психология', 'Публицистика', 'поэзия', 'публицистика', 'языкознание'
}


In [55]:
import glob
from xml.dom import minidom
import os


def get_fb2_filenames():
    return glob.iglob('/Users/kukuruku/Documents/librusec/Libruks/Архивы Либрусек/**/*.fb2', recursive=True)


def get_title_info(node):
    description = node.getElementsByTagName('title-info')
    assert len(description) == 1, 'только одно описание'
    
    return description[0]


def get_genres(node, exclude):
    genre = node.getElementsByTagName('genre')

    return {g.childNodes[0].data for g in genre if len(g.childNodes)} - exclude


def find_lang(node):
    lang = node.getElementsByTagName('lang')
    assert len(lang) <= 1, 'указано небольше одного языка'
    
    return lang[0].childNodes[0].data if len(lang) and len(lang[0].childNodes) else None

def get_title(node):
    title = node.getElementsByTagName('book-title')
    assert len(title) == 1 and len(title[0].childNodes) == 1, 'указано название книги'
    
    return title[0].childNodes[0].data



count = 0
all_genres = set()

for filename in get_fb2_filenames():
    try:
        doc = minidom.parse(filename)
        info = get_title_info(doc)
        
        count += 1

        if count % 100 == 0:
            print(count)

        genres = get_genres(info, forbidden_genres)
        lang = find_lang(info)
            
        if not len(genres):
            os.remove(filename)
            continue
            
        if not lang or lang.lower() != 'ru':
            os.remove(filename)
            print('del', lang, get_title(info))
            continue
            
        all_genres |= genres
            
#         print(genres, get_title(info))

    except Exception as e:
        print(filename, e)
        
all_genres

100
200
300
/Users/kukuruku/Documents/librusec/Libruks/Архивы Либрусек/106505-107101/106934.fb2 mismatched tag: line 154, column 2
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
/Users/kukuruku/Documents/librusec/Libruks/Архивы Либрусек/185978-186501/185993.fb2 encoding specified in XML declaration is incorrect: line 1, column 31
2000
2100
2200
2300
2400
2500
/Users/kukuruku/Documents/librusec/Libruks/Архивы Либрусек/009102-013503/9102.fb2 not well-formed (invalid token): line 360, column 1349
/Users/kukuruku/Documents/librusec/Libruks/Архивы Либрусек/031274-034596/33264.fb2 not well-formed (invalid token): line 151, column 405
/Users/kukuruku/Documents/librusec/Libruks/Архивы Либрусек/031274-034596/32433.fb2 not well-formed (invalid token): line 62, column 552
2600
2700
2800
2900
3000
/Users/kukuruku/Documents/librusec/Libruks/Архивы Либрусек/013504-016770/15136.fb2 mismatched tag: line 63, column 177
3100
3200
3300
3400
3500
3600
3700
3800
3900
/Users/kukur

{'child_4',
 'child_9',
 'child_adv',
 'child_animals',
 'child_characters',
 'child_det',
 'child_education',
 'child_history',
 'child_nature',
 'child_prose',
 'child_sf',
 'child_tale',
 'child_verse',
 'children'}

In [54]:

'l'.lower()

doc = minidom.parse('/Users/kukuruku/Documents/librusec/Libruks/Архивы Либрусек/172726-173006/172846.fb2')

# print(doc.toprettyxml())

info = get_title_info(doc)

count += 1

if count % 100 == 0:
    print(count)

get_genres(info, forbidden_genres)
lang = find_lang(info)

0